In [1]:
import numpy as np
import pandas as pd
import os
import random
import cv2
import math
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from keras.utils.np_utils import to_categorical
from skimage.transform import rotate

Using TensorFlow backend.


## Get files from a path

In [2]:
def get_files(path):
    files = list()
    
    # /kaggle/input/dataset/train
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            files.append(os.path.join(dirname, filename))
    
    return files

In [3]:
train_files_path = get_files('/kaggle/input/dance-form-vaibhav/dataset/train/')

train_csv = pd.read_csv('/kaggle/input/dance-form-vaibhav/dataset/train.csv')

print("Train files : {0}".format(len(train_files_path)))

Train files : 364


In [4]:
train_dict = dict(zip(train_csv.Image, train_csv.target))
print(train_dict['96.jpg'])

manipuri


### Split into training and validation set

In [5]:
split = int(len(train_files_path) * 0.83)
random.shuffle(train_files_path)
train_files = train_files_path[:split]
val_files = train_files_path[split:]

print("Training files {0}".format(len(train_files)))
print("Validation files {0}".format(len(val_files)))

Training files 302
Validation files 62


### Converting dance forms into numbers

In [6]:
unique_labels = list(set(train_dict.values()))
unique_labels_mapping = dict()
for index in range(len(unique_labels)):
    unique_labels_mapping[unique_labels[index]] = index

total_classes = len(unique_labels)
print("Number of classes : {0}".format(total_classes))

Number of classes : 8


In [7]:
def get_one_hot_encoded_mask(value, num_labels):
    return to_categorical(value, num_classes = num_labels)

### Setting up hyper-parameters

In [58]:
img_width, img_height = 256, 256
batch_size = 16
epochs = 10
learning_rate = 1e-3

### Creating Augmentations

In [9]:
def rotate_image(image, angle_list):
    rotated_images = list()
    for angle in angle_list:
        rotated_images.append(rotate(image,angle))
    
    return rotated_images

In [23]:
def scaleDown_image(image, fx=0.6, fy=0.6):
    return cv2.resize(image, None, fx= 0.6, fy= 0.6, interpolation= cv2.INTER_LINEAR)

In [24]:
def scaleUp_image(image, fx = 2, fy = 2):
    return cv2.resize(image, None, fx = fx, fy = fy, interpolation= cv2.INTER_LINEAR)

In [12]:
def horizontal_flip(img):
    return img[: , ::-1]

## Creating DataGenerator

In [55]:
class DataGenerator:
    def __init__(self, train_files, valid_files, labels_dict, batch_size = 16, val_augment = True):
        self.train_files = train_files
        self.valid_files = valid_files
        self.labels_dict = labels_dict
        self.batch_size = batch_size
        self.val_augment = val_augment
        
    def train_generator(self):
        num_images = len(self.train_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                x_batch = list()
                y_batch = list()
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.train_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    x_batch.append(img)
                    
                    image_name = self.train_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name[-1])]]
                    label = get_one_hot_encoded_mask(label, total_classes)
                    y_batch.append(label)
                    
                    rotated_images = rotate_image(img, [45, 60, -45, -60])
                    for rotated_image in rotated_images:
                        x_batch.append(rotated_image)
                        y_batch.append(label)
                                        
                    x_batch.append(horizontal_flip(img))
                    y_batch.append(label)
                
                yield (np.asarray(x_batch), np.asarray(y_batch))
    
    def valid_generator(self):
        num_images = len(self.valid_files)
        while True:
            x_batch = list()
            y_batch = list()
            index_list = list(range(0,num_images))
            index_list = shuffle(index_list)
            for idxs in range(0, num_images, self.batch_size):
                x_batch = list()
                y_batch = list()
                for idx in index_list[idxs:min(idxs+self.batch_size, num_images)]:
                    
                    img = cv2.imread(self.valid_files[idx])
                    img = cv2.resize(img, (img_width, img_height))
                    x_batch.append(img)
                    
                    image_name = self.valid_files[idx].split("/")
                    label = unique_labels_mapping[train_dict[str(image_name[-1])]]
                    label = get_one_hot_encoded_mask(label, total_classes)
                    y_batch.append(label)
                    
                    if self.val_augment:
                        rotated_images = rotate_image(img, [45, 60, -45, -60])
                        for rotated_image in rotated_images:
                            x_batch.append(rotated_image)
                            y_batch.append(label)

                        x_batch.append(horizontal_flip(img))
                        y_batch.append(label)
                        
                yield (np.asarray(x_batch), np.asarray(y_batch))

## Model Experiments

In [62]:
from keras.applications import vgg19
from keras.layers import Dense, GlobalAveragePooling2D, Flatten
from keras.optimizers import Adam, SGD
from keras.models import Model

### VGG16 Setup

In [15]:
model = vgg19.VGG19(include_top=False, input_shape=(img_width, img_height, 3), \
                    weights='/kaggle/input/vgg19/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [16]:
epoch_steps = int(math.ceil(len(train_files) / batch_size))
print(epoch_steps)
val_steps = int(math.ceil(len(val_files) / batch_size))
print(val_steps)

19
4


In [17]:
x = model.output
x = Flatten()(x)
x = Dense(512,activation='relu')(x)
x = Dense(256,activation='relu')(x)
final_layer = Dense(total_classes,activation='softmax')(x)

In [18]:
model=Model(inputs=model.input,outputs=final_layer)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 256, 256, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 256, 256, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 128, 128, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, 128, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, 128, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 64, 64, 128)       0     

In [59]:
model.compile(optimizer=SGD(learning_rate=learning_rate , momentum = 0.9), 
              loss='categorical_crossentropy',metrics=['accuracy'])

In [60]:
datagen = DataGenerator(train_files, val_files, train_dict, batch_size)

In [61]:
history = model.fit_generator(generator=datagen.train_generator() , steps_per_epoch=epoch_steps, 
                             epochs=10, validation_steps = val_steps, 
                             validation_data=datagen.valid_generator(), verbose=2)

Epoch 1/10
19/19 - 22s - loss: 2.0766 - accuracy: 0.1391 - val_loss: 2.0938 - val_accuracy: 0.0484
Epoch 2/10
19/19 - 23s - loss: 2.0761 - accuracy: 0.1391 - val_loss: 2.0915 - val_accuracy: 0.0645
Epoch 3/10
19/19 - 22s - loss: 2.0756 - accuracy: 0.1391 - val_loss: 2.0932 - val_accuracy: 0.0484
Epoch 4/10


KeyboardInterrupt: 